In [218]:
import msdial5_formatter.py

ModuleNotFoundError: No module named 'msdial5_formatter.py'; 'msdial5_formatter' is not a package

In [172]:
# Example usage
msdial5_formatter.convert_mgf('Mgf_2024_01_17_15_46_40_AlignmentResult_2024_01_17_14_46_28.mgf',
                        'Mgf_2024_01_17_15_46_40_AlignmentResult_2024_01_17_14_46_28_converted.mgf')


In [215]:
def convert_to_feature_csv(input_filename, output_filename):
    # Read the entire table without treating the first row as headers
    file_extension = os.path.splitext(input_filename)[-1].lower()
    if file_extension == '.csv':
        separator = ','
    elif file_extension == '.tsv' or file_extension == '.txt':
        separator = '\t'
    else:
        raise ValueError("Unsupported file extension. Supported extensions are .csv, .tsv, and .txt")

    input_format = pd.read_csv(input_filename, sep=separator, header=None)

    # Set the first row as the new header
    new_header = input_format.iloc[0]
    input_format.columns = new_header

    # Remove the original first row from the data
    input_format = input_format[1:]
    class_column_position = input_format.columns.get_loc("Class")

    # Keep all columns from the "Class" column position onwards and the first three columns
    columns_to_keep_indices = list(range(3)) + list(range(class_column_position+1, len(input_format.columns)))
    input_format = input_format.iloc[:, columns_to_keep_indices]
    input_format.columns = ['A', 'B', 'C'] + list(input_format.columns[3:])
    nan_columns = input_format.columns[input_format.columns.isna()]
    input_format = input_format.drop(columns=nan_columns)

    input_format.columns = input_format.iloc[3]  # Use the third row as header
    input_format = input_format.drop(input_format.index[:4])  # Drop the first three rows

    # Check if "Alignment ID" is in the columns
    if "Alignment ID" not in input_format.columns:
        # If not present, use iloc[2] and index[:3]
        input_format.columns = input_format.iloc[2]  # Use the third row as header
        input_format = input_format.drop(input_format.index[:3])  # Drop the first three rows

    if "Alignment ID" in input_format.columns:
        input_format["Alignment ID"] = input_format["Alignment ID"].astype(int) + 1
    
    rename_map = {
        "Alignment ID": "row ID",
        "Average Rt(min)": "row retention time",
        "Average Mz": "row m/z"
    }
    input_format = input_format.rename(columns=rename_map)
    for col in rename_map.values():
        if col in input_format.columns:
            input_format[col] = input_format[col].astype(str)

        # Append " Peak area" to the remaining column names
    for col in input_format.columns:
        if col not in rename_map.values():
            input_format = input_format.rename(columns={col: col + " Peak area"})

    columns_order = ["row ID", "row m/z", "row retention time"] + [col for col in input_format.columns if col not in ["row ID", "row m/z", "row retention time"]]
    output = input_format[columns_order]

    output.to_csv(output_filename, sep=",", index=False)

    return output

In [216]:
msdial5_formatter.convert_to_feature_csv('Height_0_2024_01_17_15_46_14.txt', 
                       'Height_0_2024_01_17_15_46_14_converted.csv')

4,row ID,row m/z,row retention time,20230627_Ben_T1_p Peak area,20230627_Ben_T10_p Peak area,20230627_Ben_T11_p Peak area,20230627_Ben_T12_p Peak area,20230627_Ben_T13_p Peak area,20230627_Ben_T14_p Peak area,20230627_Ben_T15_p Peak area,...,20230627_Ben_T8_p Peak area,20230627_Ben_T80_p Peak area,20230627_Ben_T81_p Peak area,20230627_Ben_T82_p Peak area,20230627_Ben_T83_p Peak area,20230627_Ben_T84_p Peak area,20230627_Ben_T85_p Peak area,20230627_Ben_T86_p Peak area,20230627_Ben_T87_p Peak area,20230627_Ben_T9_p Peak area
5,1,57.07291,14.793,297,496,324,224,263,258,238,...,682,26,60,55,30,98,66,70,38,898
6,2,58.03064,10.179,692,752,767,32,9,596,411,...,905,0,0,0,6,9,0,0,17,10
7,3,59.05004,5.536,680,682,518,886,604,359,624,...,1029,64,87,159,37,151,98,98,77,2512
8,4,66.02195,0.673,8174,5035,3718,3167,3032,3923,3706,...,4147,1120,1391,1155,579,1211,976,1720,861,3638
9,5,67.05531,9.471,369,811,531,532,288,283,293,...,1144,30,99,100,30,44,63,29,44,1876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3881,3877,1111.96777,16.707,36,18,8,26,29,24,22,...,31,0,0,1,1,2,3,2,2,31
3882,3878,1111.98242,16.912,29,20,23,41,22,14,10,...,13,1,0,4,1,0,3,1,0,16
3883,3879,1127.96106,17.080,28,24,31,38,28,17,28,...,53,3,0,2,0,2,2,0,0,21
3884,3880,1146.33020,15.805,659,172,459,128,82,152,120,...,297,2,3,6,2,2,10,4,4,208


In [217]:
msdial5_formatter.convert_to_feature_csv('Area_0_2024_01_17_15_46_14.txt', 
                       'Area_0_2024_01_17_15_46_14_converted.csv')

4,row ID,row m/z,row retention time,20230627_Ben_T1_p Peak area,20230627_Ben_T10_p Peak area,20230627_Ben_T11_p Peak area,20230627_Ben_T12_p Peak area,20230627_Ben_T13_p Peak area,20230627_Ben_T14_p Peak area,20230627_Ben_T15_p Peak area,...,20230627_Ben_T8_p Peak area,20230627_Ben_T80_p Peak area,20230627_Ben_T81_p Peak area,20230627_Ben_T82_p Peak area,20230627_Ben_T83_p Peak area,20230627_Ben_T84_p Peak area,20230627_Ben_T85_p Peak area,20230627_Ben_T86_p Peak area,20230627_Ben_T87_p Peak area,20230627_Ben_T9_p Peak area
5,1,57.07291,14.793,2180,3375,1970,1310,2047,1791,1447,...,3628,128,237,225,114,566,459,300,152,5352
6,2,58.03064,10.179,3129,4053,3946,108,33,3374,1956,...,5203,0,0,0,16,23,1,0,54,33
7,3,59.05004,5.536,3342,3179,2597,6184,3274,1854,3114,...,5336,269,390,596,127,861,537,556,274,13703
8,4,66.02195,0.673,46553,25900,17742,15312,14747,18856,17983,...,20218,5075,6277,5194,2636,5593,4618,7993,3811,17292
9,5,67.05531,9.471,2634,5553,3361,3644,1755,2039,1790,...,6878,142,548,543,129,181,287,155,190,11927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3881,3877,1111.96777,16.707,158,67,20,183,150,70,138,...,133,0,1,3,2,7,8,5,10,193
3882,3878,1111.98242,16.912,138,130,88,176,108,65,27,...,48,3,0,12,3,1,7,4,0,81
3883,3879,1127.96106,17.080,137,127,111,280,121,67,123,...,213,7,0,6,2,4,4,1,0,88
3884,3880,1146.33020,15.805,3383,994,2333,757,685,966,705,...,1671,4,9,26,7,7,63,18,12,1178
